In [3]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import cv2
import requests
import random
import numpy as np

SEED = 42
np.random.seed(SEED)


In [4]:
TRAIN = True
#Number of Epochs to train for.
#EPOCHS = 250
EPOCHS = 100

In [11]:
import os

directory_path = '/content/drive/MyDrive/dcustomdata/train/images'

if os.path.exists(directory_path):
    file_list = os.listdir(directory_path)
    print(file_list)
else:
    print("Directory does not exist:", directory_path)



Directory does not exist: /content/drive/MyDrive/dcustomdata/train/images


In [12]:
def download_file(url, save_name):
    url = url
    if not os.path.exists(save_name):
        file = requests.get(url)
        open(save_name, 'wb').write(file.content)
    else: 
        print('File already present, skipping download...')

In [ ]:
class_names = ['Black Spot', 'Greening', 'Ovalado', 'Scab', 'Healthy']  
colors = np.random.uniform(0, 255, size=(len(class_names), 5))

In [ ]:
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # denormalize the coordinates
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)
        width = xmax - xmin
        height = ymax - ymin
        
        class_name = class_names[int(labels[box_num])]
        
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=colors[class_names.index(class_name)],
            thickness=2
        ) 

        font_scale = min(1,max(3,int(w/500)))
        font_thickness = min(2, max(10,int(w/50)))
        
        p1, p2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))
        # Text width and height
        tw, th = cv2.getTextSize(
            class_name, 
            0, fontScale=font_scale, thickness=font_thickness
        )[0]
        p2 = p1[0] + tw, p1[1] + -th - 10
        cv2.rectangle(
            image, 
            p1, p2,
            color=colors[class_names.index(class_name)],
            thickness=-1,
        )
        cv2.putText(
            image, 
            class_name,
            (xmin+1, ymin-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            font_thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_training_images = glob.glob(image_paths)
    all_training_labels = glob.glob(label_paths)
    all_training_images.sort()
    all_training_labels.sort()
    
    num_images = len(all_training_images)
    
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image = cv2.imread(all_training_images[j])
        with open(all_training_labels[j], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=0)
    plt.tight_layout()
    plt.show()

In [ ]:
import glob

def set_res_dir():
    # Determine the directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print("Current number of result directories:", res_dir_count)

    if TRAIN:
        res_dir_count += 1

    RES_DIR = f"results_{res_dir_count}"
    print("Result directory:", RES_DIR)
    
    return RES_DIR

In [17]:
def monitor_tensorboard():
    %load_ext tensorboard
    %tensorboard --logdir /content/drive/MyDrive/yolov5/runs/train/results_1

In [ ]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15705 (delta 10), reused 17 (delta 6), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.50 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (10755/10755), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/yolov5 /content/drive/MyDrive/

In [ ]:
%cd /content/drive/MyDrive/yolov5
!pwd

/content/drive/MyDrive/yolov5
/content/drive/MyDrive/yolov5


In [ ]:
!pwd

/content/drive/MyDrive/yolov5


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 23.5 MB/s eta 0:00:00


In [18]:
monitor_tensorboard()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with -6).
Contents of stderr:
2023-05-29 08:01:10.584021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 08:01:11.414878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-29 08:01:12.807933: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-29 08:01:12.837701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative

In [1]:
#!rm -rf /content/drive/*
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
RES_DIR = set_res_dir()
if TRAIN:
    !python train.py --data /content/yolov5/data/customdata.yaml --weights yolov5s.pt \
    --img 1056 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

Current number of result directories: 0
Result directory: results_1
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/customdata.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=16, imgsz=1056, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=results_1, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=

In [ ]:
RES_DIR = set_res_dir()
if TRAIN:
    !python train.py --data /content/yolov5/data/customdata.yaml --weights yolov5s.pt \
    --img 1056 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

Current number of result directories: 3
Result directory: results_4
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/customdata.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=250, batch_size=16, imgsz=1056, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=results_4, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw

In [ ]:
RES_DIR = set_res_dir()
if TRAIN: 
    !python train.py --data /content/drive/MyDrive/dcustondata.yaml --weights yolov5s.pt \
    --img 1056 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR} \
    --freeze 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

Current number of result directories: 5
Result directory: results_6
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/dcustondata.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=1056, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=results_6, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=

In [ ]:
#model.save('runs/train/results_1/weights/last.pt')

In [ ]:
# Function to show validation predictions saved during training.
def show_valid_results(RES_DIR):
    !ls runs/train/{RES_DIR}
    EXP_PATH = f"runs/train/{RES_DIR}"
    validation_pred_images = glob.glob(f"{EXP_PATH}/*_pred.jpg")
    print(validation_pred_images)
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

In [ ]:
# Helper function for inference on images.
def inference(RES_DIR, data_path):
    # Directory to store inference results.
    infer_dir_count = len(glob.glob('runs/detect/*'))
    print(f"Current number of inference detection directories: {infer_dir_count}")
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)
    # Inference on images.
    !python detect.py --weights runs/train/{RES_DIR}/weights/best.pt \
    --source {data_path} --name {INFER_DIR}
    return INFER_DIR

In [ ]:
rnd_array = np.random.randint(0,50, size=10)

In [ ]:
def visualize(INFER_DIR):
# Visualize inference images.
    INFER_PATH = f"runs/detect/{INFER_DIR}"
    infer_images = glob.glob(f"{INFER_PATH}/*.jpg")
    print(infer_images)
    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

In [ ]:
show_valid_results(RES_DIR)

In [ ]:
# Inference on images.
IMAGE_INFER_DIR = inference(RES_DIR, '/content/drive/MyDrive/train_data/images/test')

Current number of inference detection directories: 1
inference_2
detect: weights=['runs/train/results_1/weights/best.pt'], source=/content/drive/MyDrive/train_data/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=inference_2, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/555 /content/drive/MyDrive/train_data/images/test/G (1).jpg: 640x640 1 Greening, 11.6ms
image 2/555 /content/drive/MyDrive/train_data/images/test/G (10).jpg: 640x640 1 Greening, 11.6ms
image 3/555 /content/drive/MyDrive/train_data/im

In [ ]:
visualize(IMAGE_INFER_DIR)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [ ]:
!pip install -U torch torchvision
!pip install --upgrade git+https://github.com/ultralytics/yolov5.git



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ultralytics/yolov5.git to /tmp/pip-req-build-0x78ogm_
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/yolov5.git /tmp/pip-req-build-0x78ogm_
  Resolved https://github.com/ultralytics/yolov5.git to commit c3c130416323f3766d4abe95c2ff88bc9e2264dd
ERROR: git+https://github.com/ultralytics/yolov5.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [ ]:
!pip install ultralytics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 19.7 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [ ]:
!python -m tf2onnx.--tflite --opset 11 --tflite --output yolov5s.tflite --inputs input --outputs output --inputs-as-nchw /content/drive/MyDrive/yolov5/runs/train/results_1/weights/best.onnx


/usr/bin/python3: No module named tf2onnx.--tflite


In [ ]:
!pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python export.py --weights /content/drive/MyDrive/yolov5/runs/train/results_1/weights/best.pt --include tflite --img 1056

Traceback (most recent call last):
  File "/content/drive/MyDrive/yolov5/export.py", line 71, in <module>
    from models.yolo import ClassificationModel, Detect, DetectionModel, SegmentationModel
  File "/content/drive/MyDrive/yolov5/models/yolo.py", line 24, in <module>
    from models.common import *
  File "/content/drive/MyDrive/yolov5/models/common.py", line 28, in <module>
    from utils.dataloaders import exif_transpose, letterbox
  File "/content/drive/MyDrive/yolov5/utils/dataloaders.py", line 31, in <module>
    from utils.augmentations import (Albumentations, augment_hsv, classify_albumentations, classify_transforms, copy_paste,
  File "/content/drive/MyDrive/yolov5/utils/augmentations.py", line 15, in <module>
    from utils.general import LOGGER, check_version, colorstr, resample_segments, segment2box, xywhn2xyxy
  File "/content/drive/MyDrive/yolov5/utils/general.py", line 38, in <module>
    from ultralytics.yolo.utils.checks import check_requirements
ModuleNotFoundErro

In [ ]:
!python export.py --weights /content/drive/MyDrive/yolov5/runs/train/results_1/weights/best.pt --include tflite --int8 --img 1056 --data /content/drive/MyDrive/yolov5/data/customdata.yaml


export: data=/content/drive/MyDrive/yolov5/data/customdata.yaml, weights=['/content/drive/MyDrive/yolov5/runs/train/results_1/weights/best.pt'], imgsz=[1056], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=True, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients

PyTorch: starting from /content/drive/MyDrive/yolov5/runs/train/results_1/weights/best.pt with output shape (1, 68607, 10) (14.1 MB)

TensorFlow SavedModel: starting export with tensorflow 2.12.0...

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1  